# Watson Assistant - Deploy Workspace to Github
### Omar Megawer 
#### 1-May-2018

In [ ]:
import pandas as pd
import numpy as np
from watson_developer_cloud import AssistantV1
import sys
import json
from datetime import datetime

#### Download a fresh copy of the WA workspace
Add the credentials for your workspace here.

In [ ]:
ctx = {
  "url": "https://gateway-fra.watsonplatform.net/conversation/api",
  "username": "",
  "password": ""
}

workspace_id = ""
initials = 'OM'#This is used to track who took the backup of the workspace

In [ ]:
assistant = AssistantV1( username=ctx.get('username'), password=ctx.get('password'), version='2018-02-16', url= ctx.get('url'))

In [ ]:
response = assistant.get_workspace(workspace_id=workspace_id, export=True)

In [ ]:
def writeToJSONFile(path, fileName, data):
    filePathNameWExt = './' + path + '/' + fileName + '.json'
    with open(filePathNameWExt, 'w') as fp:
        json.dump(data, fp, ensure_ascii=False)


filename = response['name'] + datetime.now().strftime('_%Y%m%d_%H%M_') +'_'+initials #You can change the name of the file as per your requirements

writeToJSONFile('./',filename,response)

#### Connect to your Github Repo

In [ ]:
import gitlab
import json

GITHUB_BASE = 'https://git.ng.bluemix.net'
GITHUB_USERID = ""
GITHUB_TOKEN = ""
GITHUB_REPO = ""
GITHUB_PROJECT = GITHUB_USERID + "/" + GITHUB_REPO

gh = gitlab.Gitlab(GITHUB_BASE, GITHUB_TOKEN)

project = gh.projects.get(GITHUB_PROJECT)

issues = project.issues.list(all=True)

##### This is to attach a file to an issue
Specify the issue number where you would like to back up your workspace.

In [ ]:
uploaded_file = project.upload(filename+'.json', filepath='./'+filename+'.json')

issue = project.issues.get(201) #This is the issue number
issue.notes.create({
    "body": "See the attached file: {}".format(uploaded_file["markdown"])
})